In [1]:
from datetime import datetime, timedelta

from dateutil import tz
from dateutil.relativedelta import relativedelta

from helper_functions import print_dts

# Introduction

**Binder Link:** https://bit.ly/2W7TEAD ([Full link](https://mybinder.org/v2/gh/pganssle-talks/pycon-us-2019-dealing-with-datetimes/master?urlpath=lab/tree/materials))
<br/><br/>

**Repository link:** https://github.com/pganssle-talks/pycon-us-2019-dealing-with-datetimes
<br/><br/>
WiFi information:<br/>
<input type="text" style="font-size: 40px; width: 60%; height: 4em;" />

Introduction: Go over who I am, how the tutorial will be organized.

This tutorial has a lot of information packed into 3 hours, and in a lot of situations, these are "best practices" that you need to internalize rather than the building blocks of some application or another. I will cover some concept, we'll have a short exercise where you try and work with the concept, then I'll work through my approach.

Each section has some Jupyter notebooks in it, we'll be taking them in order. My answers to the exercises are usually in a `.py` file alongside the notebooks.

# Overview

- Time Zones
- Serializing and deserializing datetimes
- Recurring events

# Datetime Basics: Absolute vs. Civil Times

* Absolute times ("timestamps"): A specific moment in time
    - Usually the right choice for times in the *past* (logs, etc)
    - Store as UTC or fixed offset
    
<br/>

- Civil times ("wall times"): The time it says on the clock in a given place
    - Usually the right choice for times in the *future* (meetings, alarms, etc)
    - Store as local time (+ time zone)

# Datetime Basics: `datetime.datetime`

## `datetimes` are immutable

In [2]:
dt = datetime(2019, 5, 1, 13)
try:
    dt.year = 2020
except Exception as e:
    print(repr(e))

AttributeError("attribute 'year' of 'datetime.date' objects is not writable")


Use the `replace` method to get a new `datetime` with one or more properties modified:

In [3]:
dt.replace(year=2020)

datetime.datetime(2020, 5, 1, 13, 0)

In [4]:
dt.replace(year=2020, minute=30)

datetime.datetime(2020, 5, 1, 13, 30)

# Datetime Basics: `datetime.timedelta`

- Represents the difference between two "wall times"
- Does *not* represent total elapsed time between two times

In [5]:
from datetime import timedelta

In [6]:
dt_dst = datetime(2019, 11, 2, 12, tzinfo=tz.gettz("America/New_York"))
dt_std = dt_dst + timedelta(hours=24)
print(dt_dst)
print(dt_std)

2019-11-02 12:00:00-04:00
2019-11-03 12:00:00-05:00


In [7]:
print(dt_std.astimezone(tz.UTC) - dt_dst.astimezone(tz.UTC))

1 day, 1:00:00


# Datetime Basics: Aware vs. Naive datetimes

- Naive datetime: No time zone information
- Aware datetime: Localized to a specific time zone

In [8]:
print(datetime(2019, 5, 1, 13))

2019-05-01 13:00:00


In [9]:
print(datetime(2019, 5, 1, 13, tzinfo=tz.gettz("America/New_York")))

2019-05-01 13:00:00-04:00


## Parsing

Just a sample of ways people write datetimes:

- `1994-11-05T08:15:30Z`
- `1994-11-05T14:00:30+05:45`
- `19941105T08:15:30+0000`
- `1994-11-05T081530+00:00`
- `1994-W44-6T08:15:30+00:00`
- `1994-11-05 08:15:30Z`
- `1994-11-05T08:15:30,000Z`
- `1994-11-05T08:15:30.000Z`

**... and that's just a few ISO 8601/RFC 3339 representations**

Don't forget:

- `2014 Feb 12 10:30pm`
- `2014年12月30日`
- `12:30 PM`
- `13NOV2017`
- `December.0031.30`

## Recurrences

In [10]:
from dateutil import rrule as rr

# Close of business in New York on weekdays
closing_times = rr.rrule(freq=rr.DAILY, byweekday=(rr.MO, rr.TU, rr.WE, rr.TH, rr.FR),
                         byhour=17, dtstart=datetime(2017, 3, 9, 17), count=5)

for dt in closing_times:
    print(dt.replace(tzinfo=tz.gettz('America/New_York')))

2017-03-09 17:00:00-05:00
2017-03-10 17:00:00-05:00
2017-03-13 17:00:00-04:00
2017-03-14 17:00:00-04:00
2017-03-15 17:00:00-04:00


In [11]:
for dt in closing_times:
    print(dt.replace(tzinfo=tz.gettz('America/New_York')).astimezone(tz.UTC))

2017-03-09 22:00:00+00:00
2017-03-10 22:00:00+00:00
2017-03-13 21:00:00+00:00
2017-03-14 21:00:00+00:00
2017-03-15 21:00:00+00:00


## Calendar offsets

In [12]:
from dateutil.relativedelta import SU

In [13]:
# Pins to the end of the month
datetime(2018, 1, 31) + relativedelta(months=1)

datetime.datetime(2018, 2, 28, 0, 0)

In [14]:
datetime(2018, 1, 31) + relativedelta(months=3)

datetime.datetime(2018, 4, 30, 0, 0)

In [15]:
# Get the beginning of the next month
next_month = relativedelta(months=1, day=1)
dts = [datetime(2015, 2, 1),  datetime(2015, 2, 28), datetime(2015, 3, 1)]
print_dts([(x, x + next_month) for x in dts])

               start_date               |             +relativedelta             
----------------------------------------|----------------------------------------
               2015-02-01               |               2015-03-01               
               2015-02-28               |               2015-03-01               
               2015-03-01               |               2015-04-01               


# Section 2: Time Zones

- Open `materials/02-time_zones/workbook01-python_time_zones.ipynb`

# Section 3: Serialization and Deserialization

- Open `materials/03-serializing_deserializing/workbook01-numeric.ipynb`

## Deserialization vs. Parsing User Input

- Deserialization:
  - Output of a serialization stage
  - Standardized, unambiguous format
  
- Parsing User Input:
  - Ambiguity is likely, context is important
  - Always check your assumptions
  
<br/>
<br/>
**Do not confuse one for the other!**

# Break

Take a 15 minute break!

## Back at:
<input type="text" style="font-size: 40px; width: 60%; height: 4em;">

# Section 4: Calendar Arithmetic

- Open `materials/04-calendar_arithmetic/workbook01-relativedelta.ipynb`

# Section 5: Recurring Events

- Open `materials/05-recurring_events/workbook01-introduction.ipynb`

# Q & A